In [1]:
import QuantLib as ql
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
depositoryMaturities = [ql.Period(6, ql.Months), ql.Period(12, ql.Months)]
depositoryRates = [5.25, 5.5]
# Bond rates
bondMaturities = [ql.Period(6*i, ql.Months) for i in range(3,21)] 
bondRates = [5.75, 6.0, 6.25, 6.5, 6.75, 6.80, 7.00, 7.1, 7.15, 7.2, 7.3, 7.35, 7.4, 7.5, 7.6, 7.6, 7.7, 7.8]
maturities = depositoryMaturities + bondMaturities
rates = depositoryRates + bondRates
DataFrame(zip(maturities, rates), columns=["Maturities","Curve"], index=['']*len(rates))

,Maturities,Curve
,6M,5.25
,12M,5.50
,18M,5.75
,24M,6.00
,30M,6.25
,36M,6.50
,42M,6.75
,48M,6.80
,54M,7.00
,60M,7.10


In [3]:
calcDate = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = calcDate
calendar = ql.UnitedStates()
Convention = ql.Unadjusted
dayCount = ql.ActualActual(ql.ActualActual.Bond)
end_of_month = True
settlementDays = 0
faceAmount = 1000
couponFreq = ql.Period(ql.Semiannual)
settlementDays = 0

In [4]:
depositoryHelpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)), m, settlementDays, calendar, Convention, end_of_month, dayCount)
for r, m in zip(depositoryRates, depositoryMaturities)]





In [5]:
bondHelpers = []
for r, m in zip(bondRates, bondMaturities):
    terminationDate = calcDate + m
schedule = ql.Schedule(calcDate, terminationDate, couponFreq, calendar, Convention, Convention, ql.DateGeneration.Backward, end_of_month)



In [6]:
bondHelper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(faceAmount)), settlementDays, faceAmount, schedule, [r/100.0], dayCount, Convention,
                                 )
bondHelpers.append(bondHelper)

rateHelpers = depositoryHelpers + bondHelpers




In [7]:
def spotRates(yieldcurve, day_count, calendar = ql.UnitedStates(), months = 121):
    spots = []
    tenors = []
    reference = yieldcurve.referenceDate() 
    calcDate = reference
    
    for month in range(0, months):
        yrs = month/12.0
        d = calendar.advance(reference, ql.Period(month, ql.Months))
        compounding = ql.Compounded
        freq = ql.Semiannual
        zeroRate = yieldcurve.zeroRate(yrs, compounding, freq) 
        tenors.append(yrs)
        eq_rate = zeroRate.equivalentRate(dayCount, compounding, freq, calcDate, d).rate()
        spots.append(100*eq_rate)
        
        return DataFrame(zip(tenors, spots), columns=["Maturities","Curve"], index=['']*len(tenors))
    





In [8]:
yc_LogCubicDiscount = ql.PiecewiseLogCubicDiscount(calcDate, rateHelpers, dayCount)
    
splcd = spotRates(yc_LogCubicDiscount, dayCount)
splcd.tail()

,Maturities,Curve
,0.0,0.0


In [9]:
yc_linearzero = ql.PiecewiseLinearZero(calcDate, rateHelpers, dayCount)
yc_cubiczero = ql.PiecewiseCubicZero(calcDate, rateHelpers, dayCount)
splz = spotRates(yc_linearzero, dayCount)
spcz = spotRates(yc_cubiczero, dayCount)
splz.tail()

,Maturities,Curve
,0.0,0.0
